In [2]:
import sys
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
from numpy.random import seed
from tensorflow import set_random_seed
import scipy as sp
import scipy.io as spio
from matplotlib import pyplot as plt
import matplotlib.pylab as pl
%matplotlib inline
from tensorflow.python import keras
from sklearn.model_selection import train_test_split ,KFold
from tensorflow.python.keras.models import Sequential, Model
from tensorflow.python.keras.layers import Dense, Flatten, Conv1D, Dropout, MaxPooling1D, Reshape, GlobalAveragePooling1D
from tensorflow.python.keras.utils import np_utils
from tensorflow.python.keras.layers import LeakyReLU

from numpy import mean
from numpy import std
import pandas as pd
from scipy import stats
from scipy.stats import zscore
from scipy import stats
from sklearn import metrics
import keras.backend as k
import seaborn as sns
from sklearn import preprocessing
from numpy import mean
from numpy import std

from scipy import stats
from scipy.stats import zscore
from keras.constraints import max_norm


dataset = np.loadtxt("datalast.csv",delimiter=",")

def feature_normalize(dataset):

    mu = np.mean(dataset, axis=0)
    sigma = np.std(dataset, axis=0)
    z=(dataset - mu)/sigma
    dataset=stats.zscore(z)
    return (dataset)

feature_normalize(dataset)

X = dataset[:,0:177]
y = dataset[:,178]

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.14, random_state=0)


LABELS = ["0",

          "1",

          "2",

          "3",

          "4"]

 

def show_confusion_matrix(validations, predictions):
 

    #matrix = metrics.confusion_matrix(y_test_hot, predictions)

    plt.figure(figsize=(6, 4))

    sns.heatmap(c,

                cmap="coolwarm",

                linecolor='white',

                linewidths=1,

                xticklabels=LABELS,

                yticklabels=LABELS,

                annot=True,

                fmt="d")

    plt.title("Confusion Matrix")

    plt.ylabel("True Label")

    plt.xlabel("Predicted Label")

    plt.show()
    
def counts_from_confusion(confusion):

    counts_list=[]
    
# Iterate through classes and store the counts

    for i in range(confusion.shape[0]):

        tp = confusion[i, i]
        
        fn_mask = np.zeros(confusion.shape)

        fn_mask[i, :] = 1

        fn_mask[i, i] = 0

        fn = np.sum(np.multiply(confusion, fn_mask))



        fp_mask = np.zeros(confusion.shape)

        fp_mask[:, i] = 1

        fp_mask[i, i] = 0

        fp = np.sum(np.multiply(confusion, fp_mask))
        
        tn_mask = 1 - (fn_mask + fp_mask)

        tn_mask[i, i] = 0

        tn = np.sum(np.multiply(confusion, tn_mask))



        counts_list.append({'Class': i,

                           'TP': tp,
   
                           'FN': fn,

                           'FP': fp,

                           'TN': tn})
         
        print('sensitivity', tp / tp + fn)

        print('specificity', tn / tn + fp)

    return counts_list


model = Sequential()
model.add(Conv1D(60, 10, activation='relu', input_shape=(177,1)))
model.add(Conv1D(60, 10, activation='relu'))
model.add(MaxPooling1D(2))
model.add(Dropout(0.5))
model.add(Conv1D(100, 10, activation='relu'))
model.add(Conv1D(100, 10, activation='relu'))
model.add(MaxPooling1D(2))
model.add(Dropout(0.5))
model.add(Conv1D(50, 5, activation='relu'))
model.add(GlobalAveragePooling1D())
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))
print(model.summary())


print("Number of columns in the dataframe: %i" % (X.shape[1]))
print("Number of rows in the dataframe: %i\n" % (X.shape[0]))
print("\n--- Create neural network model ---\n")
print(model.summary())
cvscores = []
j=1

#kf=KFold(n_splits=5, random_state=None, shuffle=False)

scores = list()
# summarize scores
#def summarize_results(scores):
  #  m, s = mean(scores), std(scores)
 #   print('Accuracy: %.3f%%(+/-%.3f)' % (m ,s) )

   
print("\n--- Reshape data to be accepted by Keras ---\n")
X_train = np.reshape(X_train,(X_train.shape[0], X_train.shape[1],1))
y_train = y_train.reshape(y_train.shape[0],1)
y_train_hot = np_utils.to_categorical(y_train-1, 5) # subtracting y_train-1 as keras starts num_Classes index from 0 and we had it from 1 to 5
print(X_train.shape)
print(y_train_hot.shape)

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model
print("\n--- Fit the model ---\n")
history = model.fit(X_train, y_train_hot, batch_size = 400, epochs=20,
                    #callbacks=callbacks_list,
                    validation_split=0.15, verbose=1)
#print("Avarage accuracy :" , np.mean(history))


predictions = model.predict(X_train)

y_train_hot = np.argmax(y_train_hot, axis=-1)

predictions = np.argmax(predictions, axis=-1)

c = confusion_matrix(y_train_hot, predictions)

print('Confusion matrix:\n', c)
print(counts_from_confusion(c))
show_confusion_matrix(y_train_hot, predictions)

X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
y_test = y_test.reshape(y_test.shape[0],1)

y_test_hot = np_utils.to_categorical(y_test-1, 5)

# evaluate the model
print("\n--- evaluate the model ---\n")
scores = model.evaluate(X_test, y_test_hot,verbose=1)

predictions = model.predict(X_test)

y_test_hot = np.argmax(y_test_hot, axis=-1)

predictions = np.argmax(predictions, axis=-1)

c = confusion_matrix(y_test_hot, predictions)

print('Confusion matrix:\n', c)
print(counts_from_confusion(c))
show_confusion_matrix(y_test_hot, predictions)

print("\n--- Learning curve of model training ---\n")
plt.figure()
plt.plot(history.history['acc'], "g--", label="Accuracy of training data")
plt.plot(history.history['val_acc'], "g", label="Accuracy of validation data")
plt.plot(history.history['loss'], "r--", label="Loss of training data")
plt.plot(history.history['val_loss'], "r", label="Loss of validation data")

plt.title('Model Accuracy and Loss')
plt.legend()
plt.show()

cvscores.append(scores[1] * 100)
print("Avarage accuracy :%.2f% %" , np.mean(cvscores))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_5 (Conv1D)            (None, 168, 60)           660       
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 159, 60)           36060     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 79, 60)            0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 79, 60)            0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 70, 100)           60100     
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 61, 100)           100100    
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 30, 100)           0         
__________

TypeError: unsupported operand type(s) for /: 'History' and 'int'